# Compute riskmapJNR at administrative scale

## set up parameter

set the parameters of the AOI. the AOI code should be from GAO GAUL 2015 dataset. references can be found here: https://data.apps.fao.org/catalog/dataset/gaul-codes

In [ ]:
admin_code = 133

Then set up the informations relative to the FOrest cover change:
- Global forest change: "GFC"

In [ ]:
fc_source = "GFC"

finaly set up the years of your calibration and validation period

In [ ]:
calib_start = 2015
calib_end = 2018
valid_start = 2018
valid_end = 2021

from this point no inputs are required from the user

## import libs for the computation

In [ ]:
import multiprocessing as mp

from sepal_ui import aoi
from sepal_ui import mapping as sm
from matplotlib import pyplot as plt
import numpy as n
import riskmapjnr as rmj
import rasterio as rio
from rasterio.plot import show

from component.model import FccModel
from component import scripts as cs
from component import parameter as cp
from component.message import cm

## retreive the FCC informations

In [ ]:
aoi_model = aoi.AoiModel(admin=str(admin_code))

In [ ]:
fcc_model = FccModel(
    source=fc_source,
    calib_start=calib_start,
    calib_end=calib_end,
    valid_start=valid_start,
    valid_end=valid_end,
    aoi_name=aoi_model.name,
)

In [ ]:
if fc_source == "GFC":

    fcc_data = cs.fcc_from_gfc(
        fcc_model.calib_start,
        fcc_model.calib_end,
        fcc_model.valid_start,
        fcc_model.valid_end,
    )

else:
    raise Exception(f"{fc_source} not yet supported")

In [ ]:
# add the value on a map to check results

clip_data = fcc_data.select("fcc").clip(aoi_model.feature_collection)

m = sm.SepalMap()
vi = sm.ValueInspector(m=m)
m.add_control(vi)
m.addLayer(clip_data, cp.fcc_display, cm.fcc.layer)
m.zoom_ee_object(aoi_model.feature_collection)
m

In [ ]:
# create a grid to download the images

# create a grid around the aoi
grid = cs.set_grid(aoi_model.gdf)

fig, ax = plt.subplots()
grid.plot(ax=ax, edgecolor="black")
aoi_view.model.gdf.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=2)

In [ ]:
# download the images
fc = fcc_data.select("fcc").clip(aoi_model.feature_collection)
cs.download_fcc(
    grid=grid,
    image=fc.unmask(),
    dst_dir=self.fcc_model.fcc_dir(),
    alert=self.alert,
)

## compute the riskmap

In [ ]:
ncpu = mp.cpu_count() - 2

fcc_vrt = self.fcc_model.fcc_dir() / "fcc.vrt"

results_makemap = rmj.makemap(
    fcc_file=str(fcc_vrt),
    time_interval=(fcc_model.calib_len(), fcc_model.valid_len()),
    output_dir=str(fcc_model.computation_dir()),
    clean=False,
    dist_bins=np.arange(0, 1080, step=30),
    win_sizes=np.arange(5, 100, 8),
    ncat=30,
    parallel=True,
    ncpu=ncpu,
    methods=["Equal Interval", "Equal Area"],
    csize=40,
    no_quantity_error=True,
    figsize=(6.4, 4.8),
    dpi=100,
    blk_rows=128,
    verbose=True,
)

In [ ]:
ws_hat = results_makemap["ws_hat"]
m_hat = results_makemap["m_hat"]
dist_thresh = results_makemap["dist_thresh"]
print(f"The distance theshold is {dist_thresh} m.")
print(f"The best moving window size is {ws_hat} pixels.")
print(f"The best slicing algorithm is '{m_hat}'.")

In [ ]:
fname = f"riskmap_ws{ws_hat}_{m_hat}_ev.tif"
file = fcc_model.computation_dir() / "endval" / fname

with rio.open(ifile) as src:
    show(src.read(1))